In [ ]:
library(stringr)
library(data.table)
library(dplyr)
library(patchwork)
library(data.table)
library(dplyr)
library(ggplot2)
library(forcats)
library(stringr)
library(tidyr)
library(ggpattern)

In [ ]:
source("eval_functions_refactored.R")

## Binary Traits

In [ ]:
#dir <- comparisons_binary_refactored_naomit
dir <- "comparisons_binary_refactored_subpopPCs_boot/"

#read in "nocov" results with only PCs adjusted for (no age and sex)
R2_prscs <- combine_res(dir, "*PRScs_ancestry_boot_PCagesex-adj_unrel.txt") %>% unique()
colnames(R2_prscs) <- c("phecode", "anc", "cases", "controls", "score_num", "R2", "R2_lower", "R2_upper")
prscs_cols <- data.frame(score_num = paste0("SCORE",c(1:6), "_SUM"), 
                         score = c("META_afr", "META_amr", "META_eur", "AFR_afr", "HIS_amr", "EUR_eur"))
R2_prscs <- R2_prscs %>% left_join(prscs_cols)


R2_ldpred <- combine_res(dir, "*LDpred_ancestry_boot_PCagesex-adj_unrel.txt")
colnames(R2_ldpred) <- c("phecode", "anc", "cases", "controls", "score_num", "R2", "R2_lower", "R2_upper")
ldpred_cols <- data.frame(score_num = paste0("SCORE",c(1:12), "_SUM"), 
                          score = c("METAss_AFRld_inf", "METAss_AFRld_auto", "METAss_AMRld_inf", "METAss_AMRld_auto", "METAss_EURld_inf", "METAss_EURld_auto", "AFRss_AFRld_inf", "AFRss_AFRld_auto", "HISss_AMRld_inf", "HISss_AMRld_auto", "EURss_EURld_inf", "EURss_EURld_auto"))
R2_ldpred <- R2_ldpred %>% left_join(ldpred_cols)

R2_prscsx <-  combine_res(dir, "*PRScsx_ancestry_boot_PCagesex-adj_unrel.txt") %>% unique()
colnames(R2_prscsx) <- c("phecode", "anc", "cases", "controls", "score_num", "R2", "R2_lower", "R2_upper")
R2_prscsx <- rbind(R2_prscsx %>% filter(phecode == "Phe_250_2", score_num == "SCORE5_SUM"),
                   R2_prscsx %>% filter(phecode != "Phe_250_2", score_num == "SCORE4_SUM")) %>%
                        mutate(score = "prscsx")



In [ ]:

names <- data.frame(phecode = c("Phe_411_2","Phe_411_3", "Phe_250_2", "Phe_427_21", "Phe_428_1"),
                    traits = c("Myocardial Infarction", "Angina Pectoris", "Type 2 Diabetes", "Atrial Fibrillation", "Congestive Heart Failure" ),
                    names = c("MI", "AP","T2D", "AF", "CHF"))
R2_prscs1 <- R2_prscs %>% rbind(R2_prscsx)

R2_prscs2 <- R2_prscs1 %>% separate(score, into = c("ss", "ld"), remove = F)

R2_ldpred2 <- R2_ldpred %>% 
                        separate(score, into = c("ss", "ld", "type"), remove = F) %>% 
                        mutate(ss = str_remove(ss, "ss"), ld = tolower(str_remove(ld, "ld"))) 

R2_ldpred3 <- R2_ldpred2 %>% 
                        filter(type %in% c("auto", NA)) %>% 
                        select(!type)

binary_dat <- bind_rows(R2_prscs2 %>% 
                        mutate(method =  "prscs"), 
                        R2_ldpred3 %>% 
                        mutate(method = "ldpred")) %>% 
                        mutate(score2 = paste(ss, ld, method, sep = "_" ), 
                               score1 =paste(ss, ld, sep = "_" )) %>%  
                        full_join(names) %>% 
                        select(!score_num)

write.table(binary_dat, "dat_binary_unrelated_refactored_subpopPCagesex_boot.txt", quote = F, row.names = F, sep = ";")



In [ ]:
binary <- fread("dat_binary_unrelated_refactored_subpopPCagesex_boot.txt")
binary %>% select(phecode, traits, anc, cases, controls) %>% 
            mutate(tot = cases + controls, prev = cases/tot) %>% 
            unique() %>% arrange(traits, anc)

In [ ]:
cont <- fread("dat_continuous_unrelated_refactored_subpopPCagesex_boot.txt")
cont %>% select(phecode, anc, n) %>% unique() %>% arrange(phecode, anc)

In [ ]:
#preparing data for figures

dat_fill_prscsx <- data.frame(traits = rep(c("Angina Pectoris","Atrial Fibrillation", "Congestive Heart Failure", "Myocardial Infarction", "Type 2 Diabetes"), 3), 
                  names = rep(c("AP", "AF", "CHF", "MI",  "T2D"),3))
dat_fill_prscsx$anc <- c(rep("eur",5), rep("amr",5), rep("afr",5))
dat_fill_prscsx$ss <- "prscsx"
dat_fill_prscsx$score2 <- "prscsx_NA_prscs"
dat_fill_prscsx$score1  <-"prscsx_NA"
dat_fill_prscsx$method <- "ldpred"
dat_fill_prscsx$R2 <- -2


binary_dat2 <- plyr::rbind.fill(binary_dat, dat_fill_prscsx)

dat_fill <- data.frame(traits = rep(rep(c("Angina Pectoris","Atrial Fibrillation", "Congestive Heart Failure", "Myocardial Infarction",  "Type 2 Diabetes"), 3),2),
                   names = rep(rep(c("AP","AF", "CHF", "MI", "T2D"), 3),2))  
dat_fill$anc <- rep(c(rep("eur", 5), rep("amr", 5), rep("afr", 5)),2)
dat_fill$ld <- rep(c(rep("amr", 5), rep("afr", 5)),3)
dat_fill$R2 <- rep(c(rep(0, 5), rep(-2,5)),3)
dat_fill$ss <- "fill"
dat_fill$score2 <- "fill"
dat_fill$score1  <-"fill"
dat_fill$method <- "fill"

binary_dat3 <- plyr::rbind.fill(binary_dat2, dat_fill)

In [ ]:
make_plot_binary <- function(trait, ylim){
    binary_dat3 %>% filter(traits == trait, anc %in% c("eur", "afr", "amr")) %>% 
  ggplot(aes(x = fct_relevel(score1, c("prscsx_NA", "META_afr", "META_amr", "META_eur","fill", "AFR_afr", "HIS_amr", "EUR_eur")), y = R2)) + 
  geom_col_pattern( linewidth=1, width =  0.75, aes(fill=ss, pattern_density=fct_relevel(method, c( "prscs", "ldpred")), 
                                                    color = ld), pattern_fill="white", pattern_color ="white", pattern = 'stripe', 
                                                       pattern_spacing=.05, position = position_dodge(1)) + 
  geom_errorbar(aes(ymin = R2_lower, ymax = R2_upper, group = interaction(ss, fct_relevel(method, c( "prscs", "ldpred")))) , width = 0.2, position = position_dodge(1), color = "#555555") + 
  theme_minimal() +   ylim(0, ylim) +   facet_wrap(~toupper(anc)) +
  scale_color_manual(values = c("afr" = "#7D82B8",
                                "eur"="#9ED8DB",
                                "amr"="#DF3B57"), na.value="#F2CCC3") +  
  scale_fill_manual(values = c("AFR" = "#7D82B8",
                                "EUR"="#9ED8DB",
                                "HIS"="#DF3B57",
                                "META" = "#1D3354",
                                "prscsx" = "#F2CCC3"), na.value="white") + 
  scale_pattern_density_discrete(name = "Method") +   
  theme(legend.key.size = unit(1, 'cm'),panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(), axis.text.x=element_blank(), 
        axis.ticks.x=element_blank(), strip.text.x= element_blank()) + xlab(NULL) }

In [ ]:
fig_dir <- "figs_refactored_subpopPCs"
dir.create(fig_dir)

In [ ]:
names <- data.frame(phecode = c("Phe_411_2","Phe_411_3", "Phe_250_1", "Phe_250_2", "Phe_427_21", "Phe_428_1"),
                    traits = c("Myocardial Infarction", "Angina Pectoris", "Type 1 Diabetes", "Type 2 Diabetes", "Atrial Fibrillation", "Congestive Heart Failure" ),
                    names = c("MI", "AP", "T1D", "T2D", "AF", "CHF"))
ap <- make_plot_binary("Angina Pectoris", 0.031)
af <- make_plot_binary("Atrial Fibrillation", 0.066)
chf <- make_plot_binary("Congestive Heart Failure", 0.03)
mi <- make_plot_binary("Myocardial Infarction", 0.04)
t2d <- make_plot_binary("Type 2 Diabetes", 0.115)
binary_plot <- plot_spacer()/ ap / plot_spacer()/af/plot_spacer()/chf/plot_spacer()/mi/plot_spacer()/t2d + plot_layout(guides = "collect", heights = c(1, 5, 1, 5, 1, 5, 1, 5, 1, 5), nrow = 10)
ggsave(binary_plot, file=paste0(fig_dir, "/binary_plot_5_unrelated_PCagesex_95ci.pdf"), width=10, height=10, dpi=300)


## Continuous Traits

In [ ]:
#dir <- "comparisons_continuous_refactored_inormalfixed/"
dir <- "comparisons_continuous_refactored_subpopPCs_boot/"

R2_ldpred_adj <- combine_res(dir, "*LDpred_ancestry_boot_PCagesex-adj_unrel.txt") %>% distinct()
colnames(R2_ldpred_adj) <- c("phecode", "anc", "n",  "score_num", "R2", "R2_lower", "R2_upper")
ldpred_cols <- data.frame(score_num = paste0("score",c(1:12)), 
                          score = c("METAss_AFRld_inf", "METAss_AFRld_auto", "METAss_AMRld_inf", "METAss_AMRld_auto", "METAss_EURld_inf", "METAss_EURld_auto", "AFRss_AFRld_inf", "AFRss_AFRld_auto", "HISss_AMRld_inf", "HISss_AMRld_auto", "EURss_EURld_inf", "EURss_EURld_auto"))
R2_ldpred_adj <- R2_ldpred_adj %>% left_join(ldpred_cols)

R2_prscs_adj <- combine_res(dir, "*PRScs_ancestry_boot_PCagesex-adj_unrel.txt") %>% distinct()
colnames(R2_prscs_adj) <- c("phecode", "anc", "n", "score_num", "R2", "R2_lower", "R2_upper")
prscs_cols <- data.frame(score_num = paste0("score",c(1:6)), 
                         score = c("META_afr", "META_amr", "META_eur", "AFR_afr", "HIS_amr", "EUR_eur"))
R2_prscs_adj <- R2_prscs_adj %>% left_join(prscs_cols)


R2_prscsx_adj <- combine_res(dir, "*PRScsx_ancestry_boot_PCagesex-adj_unrel.txt") %>% distinct()
colnames(R2_prscsx_adj) <- c("phecode", "anc", "n", "score_num", "R2", "R2_lower", "R2_upper")
R2_prscsx_adj <- R2_prscsx_adj %>% filter(score_num == "score5") %>% mutate(score = "prscsx")

#colnames(R2_ldpred_adj)<-  c("phecode", "anc", "N", "METAss_AFRld_inf", "METAss_AFRld_auto", "METAss_AMRld_inf", "METAss_AMRld_auto", "METAss_EURld_inf", "METAss_EURld_auto", "AFRss_AFRld_inf", "AFRss_AFRld_auto", "HISss_AMRld_inf", "HISss_AMRld_auto", "EURss_EURld_inf", "EURss_EURld_auto")
#colnames(R2_prscs_adj)<-  c("phecode", "anc", "N", "META_afr", "META_amr", "META_eur", "AFR_afr", "HIS_amr", "EUR_eur")
#colnames(R2_prscsx_adj)<-  c("phecode", "anc", "N","AFRx", "EURx", "AMRx", "EASx", "prscsx")

#R2_prscs_adj <- full_join(R2_prscs_adj, R2_prscsx_adj %>% select(phecode, anc, N, prscsx))



In [ ]:
R2_ldpred_adj2 <- R2_ldpred_adj %>% 
                    separate(score, into = c("ss", "ld", "type"), remove = F) %>% 
                    mutate(ss = str_remove(ss, "ss"), ld = tolower(str_remove(ld, "ld"))) %>% 
                    filter(type == "auto")

R2_prscs_adj2 <- R2_prscs_adj %>% rbind(R2_prscsx_adj) %>% 
                    separate(score, into = c("ss", "ld"), remove = F)

dat_cont <- bind_rows(R2_prscs_adj2 %>% 
                 mutate(method =  "prscs"), 
                 R2_ldpred_adj2 %>% 
                 mutate(method = "ldpred"))

dat_cont <- dat_cont %>% mutate(score2 = paste(ss, ld, method, sep = "_" ), score1 =paste(ss, ld, sep = "_" ))
write.table(dat_cont, "dat_continuous_unrelated_refactored_subpopPCagesex_boot.txt", quote = F, row.names = F)

In [ ]:
dat_fill_prscsx <- data.frame(phecode = rep(c("mean_BMI_INT", "Hemoglobin_Mean_INT", "mean_Height_INT", "mean_Systolic_INT", "LDLC_Mean_INT"), 3))
dat_fill_prscsx$anc <- c(rep("eur",5), rep("amr",5), rep("afr",5))
dat_fill_prscsx$ss <- "prscsx"
dat_fill_prscsx$score2 <- "prscsx_NA_prscs"
dat_fill_prscsx$score1  <-"prscsx_NA"
dat_fill_prscsx$method <- "ldpred"
dat_fill_prscsx$R2 <- -2
dat_fill_prscsx$R2_lower <- -2
dat_fill_prscsx$R2_upper <- -2

#dat2 <- data.frame(phecode = c("basophils"), ss = c("prscsx"), anc = c("eur"), score2 = c("prscsx_NA_prscs"), score1 = c("prscsx_NA"), ≈ R2 = c(0))
dat_cont1 <- plyr::rbind.fill(dat_cont, dat_fill_prscsx)

dat_fill <- data.frame(phecode = rep(rep(c("mean_BMI_INT", "Hemoglobin_Mean_INT", "mean_Height_INT", "mean_Systolic_INT", "LDLC_Mean_INT"), 2),3))
dat_fill$anc <- c(rep("eur",5), rep("amr",5), rep("afr",5))
dat_fill$ss <- "fill"
dat_fill$score2 <- "fill"
dat_fill$score1  <-"fill"
dat_fill$method <- "fill"
dat_fill$ld <- rep(c(rep("amr", 5), rep("afr", 5)),3)
dat_fill$R2 <- rep(c(rep(0, 5), rep(-2, 5)),3)

#dat2 <- data.frame(phecode = c("basophils"), ss = c("prscsx"), anc = c("eur"), score2 = c("prscsx_NA_prscs"), score1 = c("prscsx_NA"), ≈ R2 = c(0))
dat_cont2 <- plyr::rbind.fill(dat_cont1, dat_fill)
names <- data.frame(phecode = c("mean_BMI_INT", "Hemoglobin_Mean_INT", "mean_Height_INT", "mean_Systolic_INT", "LDLC_Mean_INT"),
                    traits = c("BMI", "Hemoglobin",  "Height", "SBP", "LDL" ))
dat_cont2 <- dat_cont2 %>% inner_join(names)

In [ ]:
names <- data.frame(phecode = c("mean_BMI_INT", "Hemoglobin_Mean_INT", "mean_Height_INT", "mean_Systolic_INT", "LDLC_Mean_INT"),
                    traits = c("BMI", "Hemoglobin",  "Height", "SBP", "LDL" ))
dat_cont <- dat_cont %>% inner_join(names)


In [ ]:
make_plot_cont <- function(trait, ylim){
    dat_cont2 %>% filter(traits == trait, anc %in% c("eur", "afr", "amr")) %>% 
  ggplot(aes(x = fct_relevel(score1, c("prscsx_NA", "META_afr", "META_amr", "META_eur","fill", "AFR_afr", "HIS_amr", "EUR_eur")), y = R2)) + 
  geom_col_pattern( linewidth=1, width =  0.75, aes(fill=ss, pattern_density=fct_relevel(method, c( "prscs", "ldpred")), 
                                                    color = ld), pattern_fill="white", pattern_color ="white", pattern = 'stripe', 
                                                       pattern_spacing=.05, position = position_dodge(1)) + 
  #geom_errorbar(aes(ymin = R2_lower, ymax = R2_upper, group = interaction(ss, fct_relevel(method, c( "prscs", "ldpred")), ld)) , width = 0.2, position = position_dodge(1)) + 
   geom_errorbar(aes(ymin = R2_lower, ymax = R2_upper, group = interaction(ss, fct_relevel(method, c( "prscs", "ldpred")))) , width = 0.2, position = position_dodge(1), color = "#555555") + 

  theme_minimal() + ylim(0, ylim) +   facet_wrap(~toupper(anc)) +
  scale_color_manual(values = c("afr" = "#7D82B8",
                                "eur"="#9ED8DB",
                                "amr"="#DF3B57"), na.value="#F2CCC3") +  
  scale_fill_manual(values = c("AFR" = "#7D82B8",
                                "EUR"="#9ED8DB",
                                "HIS"="#DF3B57",
                                "META" = "#1D3354",
                                "prscsx" = "#F2CCC3"), na.value="white") + 
  scale_pattern_density_discrete(name = "Method") + 
  theme(legend.key.size = unit(1, 'cm'),panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(), axis.text.x=element_blank(), 
        axis.ticks.x=element_blank(), strip.text.x= element_blank()) + xlab(NULL) }

In [ ]:
bmi <- make_plot_cont("BMI", 0.12)
hemo <- make_plot_cont("Hemoglobin", 0.026)
height <- make_plot_cont("Height", 0.29)
ldl <- make_plot_cont("LDL", 0.11)
sbp <- make_plot_cont("SBP", 0.055)
cont_plot <- plot_spacer()/ bmi / plot_spacer()/hemo/plot_spacer()/height/plot_spacer()/ldl/plot_spacer()/sbp + plot_layout(guides = "collect", heights = c(1, 5, 1, 5, 1, 5, 1, 5, 1, 5), nrow = 10)
ggsave(cont_plot, file=paste0(fig_dir, "/cont_plot_5_unrel_PCagesex_95ci.pdf"), width=10, height=10, dpi=300)

In [ ]:
### Presentation Figures

### Comparison Figures

In [ ]:
##read in META bootstrap files

In [ ]:
bin_dir <- "comparisons_binary_refactored_subpopPCs_boot/"
R2_META_bin <- rbind(combine_res(bin_dir, "*PRScs_ancestry_META_boot_PCagesex-adj_unrel.txt"), 
                     combine_res(bin_dir, "*LDpred_ancestry_META_boot_PCagesex-adj_unrel.txt"))
colnames(R2_META_bin) <- c("phecode", "anc", "cases", "controls", "method", "R2", "R2_lower", "R2_upper")
R2_META_bin$method <- ifelse(R2_META_bin$method == "PRScs", "prscs", "ldpred")


In [ ]:
cont_dir <- "comparisons_continuous_refactored_subpopPCs_boot/"
R2_META_cont <- rbind(combine_res(cont_dir, "*PRScs_ancestry_META_boot_PCagesex-adj_unrel.txt"), 
                     combine_res(cont_dir, "*LDpred_ancestry_META_boot_PCagesex-adj_unrel.txt"))
colnames(R2_META_cont) <- c("phecode", "anc", "N", "method", "R2", "R2_lower", "R2_upper")
R2_META_cont$method <- ifelse(R2_META_cont$method == "PRScs", "prscs", "ldpred")

In [ ]:
a <- fread("dat_continuous_unrelated_refactored_subpopPCagesex_boot.txt")
a1 <- fread("dat_binary_unrelated_refactored_subpopPCagesex_boot.txt") 

#### PRS-CSx v ancestry-spec

In [ ]:
b <- a %>% 
        filter(anc %in% c("afr", "amr","eur"), phecode != "basophils") %>% 
        select(!c(n, score2, score_num)) %>% mutate(type = "cont") 
b1 <- a1 %>% 
        filter(anc %in% c("afr", "amr","eur"), phecode != "Phe_250_1") %>% 
        select(!c(cases, controls, score2, traits, names)) %>% 
        mutate(type = "binary")

c_x_vanc <- rbind(b,b1) %>% 
        filter(ss != "META", 
               ss == "prscsx" | anc == "afr" & grepl("AFR", ss) | anc == "eur" & grepl("EUR", ss) | anc == "amr" & grepl("HIS", ss)) %>%  
        mutate(score = ifelse(score == "prscsx", "prscsx", 
                              ifelse(method == "ldpred", "anc_specL", "anc_specP")))


In [ ]:
make_plotx_vanc <- function(trait, ylim, name){
    c_x_vanc %>% filter(phecode == trait, anc %in% c("eur", "afr", "amr")) %>% 
    ggplot(aes(x = fct_relevel(score, c("prscsx", "anc_specP", "anc_specL")), y = R2)) + 
    geom_col_pattern( linewidth=1, width =  0.75, 
                     aes(fill=ss, pattern_density=fct_relevel(method, c( "prscs", "ldpred")), 
                         color = ld), pattern_fill="white", pattern_color ="white", pattern = 'stripe', 
                         pattern_spacing=0.15, position = position_dodge(1)) +  #pattern_spacing=.05
    geom_errorbar(aes(ymin = R2_lower, ymax = R2_upper, 
                      group = interaction(ss, fct_relevel(method, c( "prscs", "ldpred")))) , 
                  width = 0.2, position = position_dodge(1), color = "#555555") + 

    theme_minimal() + 
    facet_wrap(~anc) +
    scale_color_manual(values = c("afr" = "#7D82B8",
                                "eur"="#9ED8DB",
                                "amr"="#DF3B57"), na.value="#F2CCC3") +  
    scale_fill_manual(values = c("AFR" = "#7D82B8",
                                "EUR"="#9ED8DB",
                                "HIS"="#DF3B57",
                                "META" = "#1D3354",
                                "prscsx" = "#F2CCC3"), na.value="white") + 
    scale_pattern_density_discrete(name = "Method") +  
    ylim(0, ylim) + 
    theme(plot.title = element_text(size=10), 
          legend.key.size = unit(1, 'cm'),
          panel.grid.major.x = element_blank(),
          panel.grid.minor.x = element_blank(), 
          axis.text.x=element_blank(), 
          axis.ticks.x=element_blank(), 
          legend.position = "none") + xlab(NULL) }


In [ ]:
height_XN <- make_plotx_vanc("mean_Height_INT", 0.29, "Height")
bmi_XN <- make_plotx_vanc("mean_BMI_INT", 0.12, "BMI")
hemo_XN <- make_plotx_vanc("Hemoglobin_Mean_INT", 0.026, "Hemoglobin")
ldl_XN <- make_plotx_vanc("LDLC_Mean_INT", 0.11, "LDL Cholesterol")
sbp_XN <- make_plotx_vanc("mean_Systolic_INT", 0.055, "Systolic Blood Pressure")

#cont_plot_meta_v_anc <- plot_spacer()/ bmi / plot_spacer()/hemo/plot_spacer()/height/plot_spacer()/ldl/plot_spacer()/sbp + plot_layout(guides = "collect", heights = c(1, 5, 1, 5, 1, 5, 1, 5, 1, 5), nrow = 10)
cont_plot_x_v_anc <-  bmi_XN / hemo_XN/height_XN/ldl_XN/sbp_XN + plot_layout(guides = "collect")


#cont_plot_x_v_anc <- plot_spacer()/ bmi / plot_spacer()/hemo/plot_spacer()/height/plot_spacer()/ldl/plot_spacer()/sbp + plot_layout(guides = "collect", heights = c(1, 5, 1, 5, 1, 5, 1, 5, 1, 5), nrow = 10)
#cont_plot_x_v_anc
ggsave(cont_plot_x_v_anc, file= paste0(fig_dir, "/cont_plot_5_unrel_x_v_anc_taller.pdf"), width = 3, height=10, dpi=300)

#cont_plot_x_v_anc_wide <- bmi + hemo + height + ldl + sbp + plot_layout(ncol = 5)
#cont_plot_x_v_anc_wide
#ggsave(cont_plot_x_v_anc_wide, file="cont_plot_5_unrel_x_v_anc_wide.pdf", width = 15, height= 3, dpi=500)

In [ ]:
t2d_XN <- make_plotx_vanc("Phe_250_2", 0.115, "Type 2 Diabetes") #Type 2 Diabetes
mi_XN <- make_plotx_vanc("Phe_411_2", 0.04, "Myocardial Infarction") #Myocardial Infarction
ap_XN <- make_plotx_vanc("Phe_411_3", 0.031, "Angina Pectoris")  #Angina Pectoris
af_XN <- make_plotx_vanc("Phe_427_21",  0.066, "Atrial Fibrillation") #AF
chf_XN <- make_plotx_vanc("Phe_428_1", 0.03, "Congestive Heart Failure") #CHF

bin_plot_x_v_anc <- ap_XN / af_XN/chf_XN/mi_XN/t2d_XN + plot_layout(guides = "collect")
#bin_plot_x_v_anc <- plot_spacer()/ ap / plot_spacer()/af/plot_spacer()/chf/plot_spacer()/mi/plot_spacer()/t2d + plot_layout(guides = "collect", heights = c(1, 5, 1, 5, 1, 5, 1, 5, 1, 5), nrow = 10)
#cont_plot_x_v_anc
ggsave(bin_plot_x_v_anc, file=paste0(fig_dir,"/bin_plot_5_unrel_x_v_anc_taller.pdf"), width = 3, height=10, dpi=300)




#### META v ancestry-spec

In [ ]:
colnames(R2_META_bin)

In [ ]:
b <-  a %>% filter(anc %in% c("afr", "amr","eur"), phecode != "basophils")  %>% 
        filter(ss != "prscsx") %>%    
        filter(anc == "afr" & grepl("AFR", ss) |
               anc == "eur" & grepl("EUR", ss) | 
               anc == "amr" & grepl("HIS", ss) ) %>% 
        select(phecode, anc, R2, R2_lower, R2_upper, ss, method) %>%
        rbind(R2_META_cont %>% mutate(ss = "META") %>% select(!N))

b1 <-  a1 %>% filter(anc %in% c("afr", "amr","eur"), phecode != "Phe_250_1")  %>% 
        filter(ss != "prscsx") %>% 
        filter(anc == "afr" & grepl("AFR", ss) | 
               anc == "eur" & grepl("EUR", ss) | 
               anc == "amr" & grepl("HIS", ss) ) %>%
        select(phecode, anc, method, R2, R2_lower, R2_upper, ss) %>%
        rbind(R2_META_bin %>% mutate(ss = "META") %>% select(!c(cases, controls)))


c_MN <- rbind(b, b1)

In [ ]:
make_plotmeta_vanc <- function(trait, lim, name){
    c_MN %>% mutate(score1 = ifelse(ss == "META", "META", "anc_spec"), 
                    score = paste0(score1, method)) %>% 
    filter(phecode == trait) %>%
    ggplot(aes(x = fct_relevel(score, c("METAprscs", "METAldpred",  "anc_specprscs", "anc_specldpred")), 
               y = R2)) + 
    geom_col_pattern(linewidth=1, 
                     width =  0.75, 
                     aes(fill=ss, 
                         pattern_density=fct_relevel(method, c( "prscs", "ldpred")), 
                         color = ss), 
                     pattern_fill="white", 
                     pattern_color ="white", 
                     pattern = 'stripe', 
                     pattern_spacing=0.15, position = position_dodge(1)) +  
    geom_errorbar(aes(ymin = R2_lower, ymax = R2_upper, group = interaction(ss, fct_relevel(method, c( "prscs", "ldpred")))) , width = 0.2, position = position_dodge(1), color = "#555555") + 

    theme_minimal() + 
    facet_wrap(~anc)+ 
    scale_color_manual(values = c("AFR" = "#7D82B8",
                                  "EUR"="#9ED8DB",
                                  "HIS"="#DF3B57"), 
                       na.value="#1D3354") +  
    scale_fill_manual(values = c("AFR" = "#7D82B8",
                                 "EUR"="#9ED8DB",
                                 "HIS"="#DF3B57",
                                 "META" = "#1D3354",
                                 "prscsx" = "#F2CCC3"), 
                      na.value="white") + 
    scale_pattern_density_discrete(name = "Method") +  
    ylim(0, lim) +
    theme(plot.title = element_text(size=10), 
          legend.key.size = unit(1, 'cm'),
          panel.grid.major.x = element_blank(),
          panel.grid.minor.x = element_blank(), 
          axis.text.x=element_blank(), 
          axis.ticks.x=element_blank(),
          legend.position = "none") + 
    xlab(NULL)}

In [ ]:
height_MN <- make_plotmeta_vanc("mean_Height_INT", 0.29, "Height")
bmi_MN  <- make_plotmeta_vanc("mean_BMI_INT", 0.12, "BMI")
hemo_MN  <- make_plotmeta_vanc("Hemoglobin_Mean_INT", 0.026, "Hemoglobin")
ldl_MN  <- make_plotmeta_vanc("LDLC_Mean_INT", 0.11, "LDL Cholesterol")
sbp_MN  <- make_plotmeta_vanc("mean_Systolic_INT", 0.055, "Systolic Blood Pressure")

#cont_plot_meta_v_anc <- plot_spacer()/ bmi / plot_spacer()/hemo/plot_spacer()/height/plot_spacer()/ldl/plot_spacer()/sbp + plot_layout(guides = "collect", heights = c(1, 5, 1, 5, 1, 5, 1, 5, 1, 5), nrow = 10)
cont_plot_meta_v_anc <-  bmi_MN  / hemo_MN /height_MN /ldl_MN /sbp_MN  + plot_layout(guides = "collect")

#cont_plot_meta_v_anc
ggsave(cont_plot_meta_v_anc, file=paste0(fig_dir,"/cont_plot_5_meta_v_anc_taller.pdf"), width=3, height=10, dpi=300)

#cont_plot_meta_v_anc_wide <- bmi + hemo + height + ldl + sbp + plot_layout(ncol = 5)
#cont_plot_x_v_anc_wide
#ggsave(cont_plot_meta_v_anc_wide, file="cont_plot_5_unrel_meta_v_anc_wide.pdf", width = 15, height= 3, dpi=500)

In [ ]:
t2d_MN  <- make_plotmeta_vanc("Phe_250_2", 0.115, "Type 2 Diabetes") #Type 2 Diabetes
mi_MN  <- make_plotmeta_vanc("Phe_411_2", 0.04, "Myocardial Infarction") #Myocardial Infarction
ap_MN  <- make_plotmeta_vanc("Phe_411_3", 0.031, "Angina Pectoris")  #Angina Pectoris
af_MN  <- make_plotmeta_vanc("Phe_427_21",  0.066, "Atrial Fibrillation") #AF
chf_MN  <- make_plotmeta_vanc("Phe_428_1", 0.03, "Congestive Heart Failure") #CHF

#bin_plot_meta_v_anc <- plot_spacer()/ ap / plot_spacer()/af/plot_spacer()/chf/plot_spacer()/mi/plot_spacer()/t2d + plot_layout(guides = "collect", heights = c(1, 5, 1, 5, 1, 5, 1, 5, 1, 5), nrow = 10)

bin_plot_meta_v_anc <- ap_MN  / af_MN /chf_MN /mi_MN /t2d_MN  + plot_layout(guides = "collect")
#cont_plot_x_v_anc
ggsave(bin_plot_meta_v_anc, file=paste0(fig_dir,"/bin_plot_5_meta_v_anc_taller.pdf"), width = 3, height=10, dpi=300)


#### META v prscsx

In [ ]:
c <- a %>% 
        filter(anc %in% c("afr", "amr","eur"), phecode != "basophils") %>%
        filter(ss == "prscsx") %>% 
        select(phecode, anc, R2, R2_lower, R2_upper, ss, method) %>%
        rbind(R2_META_cont %>% mutate(ss = "META") %>% select(!N))

c1 <- a1 %>% 
        filter(anc %in% c("afr", "amr","eur"), phecode != "Phe_250_1") %>% 
        filter(ss == "prscsx") %>% 
        select(phecode, anc, method, R2, R2_lower, R2_upper, ss) %>%
        rbind(R2_META_bin %>% mutate(ss = "META") %>% select(!c(cases, controls)))


c_xmeta <- rbind(c, c1) %>% 
        filter(ss == "META" | 
               ss == "prscsx") %>%  
        mutate(method = ifelse(method == "prscsx", "prscs", method)) 

In [ ]:
table(R2_META_cont$phecode)

In [ ]:
make_plotmeta_vprscsx <- function(trait, lim, name){
    c_xmeta %>% 
        mutate(score1 = ifelse(ss == "META", "META", "prscsx"), 
               score = paste0(score1, method)) %>% 
        filter(phecode == trait) %>%
    ggplot(aes(x = fct_relevel(score, c("prscsxprscs", "METAprscs", "METAldpred")), 
               y = R2)) + 
    geom_col_pattern(linewidth=1, 
                     width = 0.75, 
                     aes(fill=ss, 
                         pattern_density=fct_relevel(method, c( "prscs", "ldpred")), 
                         color = ss), 
                     pattern_fill="white", 
                     pattern_color ="white", 
                     pattern = 'stripe',
                     pattern_spacing=0.15,
                     position = position_dodge(1)) +  
    geom_errorbar(aes(ymin = R2_lower, ymax = R2_upper, 
                      group = interaction(ss, fct_relevel(method, c( "prscs", "ldpred")))) , 
                  width = 0.2, position = position_dodge(1), color = "#555555") + 
    theme_minimal() + 
    facet_wrap(~anc)+ 
    scale_color_manual(values = c("AFR" = "#7D82B8",
                                "EUR"="#9ED8DB",
                                "HIS"="#DF3B57", "META" = "#1D3354"), 
                       na.value="#F2CCC3") +  
    scale_fill_manual(values = c("AFR" = "#7D82B8",
                                "EUR"="#9ED8DB",
                                "HIS"="#DF3B57",
                                "META" = "#1D3354",
                                "prscsx" = "#F2CCC3"), 
                      na.value="white") + 
  scale_pattern_density_discrete(name = "Method") +  ylim(0, lim) + 
  theme(plot.title = element_text(size=10), legend.key.size = unit(1, 'cm'), panel.grid.major.x = element_blank(), 
        panel.grid.minor.x = element_blank(), axis.text.x=element_blank(),axis.ticks.x=element_blank(), legend.position = "none")  + xlab(NULL)
}

In [ ]:
height_MX <- make_plotmeta_vprscsx("mean_Height_INT", 0.29, "Height")
bmi_MX <- make_plotmeta_vprscsx("mean_BMI_INT", 0.12,"BMI")
hemo_MX <- make_plotmeta_vprscsx("Hemoglobin_Mean_INT", 0.026, "Hemoglobin")
ldl_MX <- make_plotmeta_vprscsx("LDLC_Mean_INT", 0.11, "LDL Cholesterol")
sbp_MX <- make_plotmeta_vprscsx("mean_Systolic_INT", 0.055, "Systolic Blood Pressure")


cont_plot_meta_v_x <-  bmi_MX / hemo_MX/height_MX/ldl_MX/sbp_MX + plot_layout(guides = "collect")
#cont_plot_meta_v_x
ggsave(cont_plot_meta_v_x, file=paste0(fig_dir,"/cont_plot_5_meta_v_x_taller.pdf"), width=3, height=10, dpi=300)

#cont_plot_meta_v_x_wide <- bmi + hemo + height + ldl + sbp + plot_layout(ncol = 5)
#cont_plot_x_v_anc_wide
#ggsave(cont_plot_meta_v_x_wide, file="cont_plot_5_unrel_meta_v_x_wide.pdf", width = 15, height= 3, dpi=500)

In [ ]:
t2d_MX <- make_plotmeta_vprscsx("Phe_250_2", 0.115, "Type 2 Diabetes") #Type 2 Diabetes
mi_MX <- make_plotmeta_vprscsx("Phe_411_2", 0.04, "Myocardial Infarction") #Myocardial Infarction
ap_MX <- make_plotmeta_vprscsx("Phe_411_3", 0.031, "Angina Pectoris")  #Angina Pectoris
af_MX <- make_plotmeta_vprscsx("Phe_427_21",  0.066, "Atrial Fibrillation") #AF
chf_MX <- make_plotmeta_vprscsx("Phe_428_1", 0.03, "Congestive Heart Failure") #CHF

#bin_plot_meta_v_x <- plot_spacer()/ ap / plot_spacer()/af/plot_spacer()/chf/plot_spacer()/mi/plot_spacer()/t2d + plot_layout(guides = "collect", heights = c(1, 5, 1, 5, 1, 5, 1, 5, 1, 5), nrow = 10)
bin_plot_meta_v_x <- ap_MX / af_MX/chf_MX/mi_MX/t2d_MX + plot_layout(guides = "collect")
#cont_plot_x_v_anc
ggsave(bin_plot_meta_v_x, file=paste0(fig_dir,"/bin_plot_5_meta_v_x_taller.pdf"), width = 3, height=10, dpi=300)

In [ ]:
make_plotmeta_vprscsx("Phe_428_1", 0.03, "Congestive Heart Failure")

In [ ]:
#all_plot_x_v_anc <- (ap + bmi) / (af + hemo)/(chf + height)/(mi + ldl)/(t2d + sbp) + plot_layout(guides = "collect")

all_plot_x_v_anc <- (ap_XN + bmi_XN) / (af_XN + hemo_XN)/(chf_XN + height_XN)/(mi_XN + ldl_XN)/(t2d_XN + sbp_XN) + plot_layout(guides = "collect")

ggsave(all_plot_x_v_anc, file=paste0(fig_dir,"all_plot_5_unrel_x_v_anc_taller_named.pdf"), width = 7, height=9, dpi=300)

all_plot_META_v_x <- (ap_MX + bmi_MX) / (af_MX + hemo_MX)/(chf_MX + height_MX)/(mi_MX + ldl_MX)/(t2d_MX + sbp_MX) + plot_layout(guides = "collect")

all_plot_META_v_anc <- (ap_MN + bmi_MN)/(af_MN + hemo_MN)/(chf_MN + height_MN)/(mi_MN + ldl_MN)/(t2d_MN + sbp_MN) + plot_layout(guides = "collect")

ggsave(all_plot_META_v_x, file=paste0(fig_dir,"/all_plot_5_unrel_META_v_x_taller_named.pdf"), width = 7, height=9, dpi=300)

ggsave(all_plot_META_v_anc, file=paste0(fig_dir,"/all_plot_5_unrel_META_v_anc_taller_named.pdf"), width = 7, height=9, dpi=300)


In [ ]:
a <- height_MN + height_XN   + height_MX  + plot_layout(guides = "collect", ncol = 3)  + plot_annotation(tag_levels = 'A')
ggsave(a, file=paste0(fig_dir,"/height_all.pdf"), width=10, height=5, dpi=300)

In [ ]:
b <- height_XN   /  height_MN / height_MX  + plot_layout(guides = "collect", nrow = 3)  + plot_annotation(tag_levels = 'A')
ggsave(b, file=paste0(fig_dir,"/height_all_tall.pdf"), width=5, height=7, dpi=300)